pca_w2v_250features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/w2v_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,7.502268,-10.692819,-0.404433,-1.043061,-3.365657,7.976577,-0.905798,0.075209,2.911449,...,1.057144e-07,-1.960831e-08,3.438281e-08,-2.237564e-08,-2.577251e-08,3.101031e-08,4.925576e-08,1.060432e-08,-5.321290e-08,-2.239432e-08
1,1,2.932372,-1.768247,-1.257079,-2.001190,-2.707743,4.679531,-1.493178,-1.619118,2.283399,...,3.790522e-08,-2.500214e-08,-2.884480e-08,-1.528578e-08,-6.651166e-08,5.620558e-08,-1.265269e-08,-7.747284e-08,-7.012600e-08,3.852026e-09
2,2,-3.811744,2.214510,5.168598,-1.560522,-4.774682,0.298724,1.614868,-0.184465,-2.296425,...,3.322145e-08,-7.995277e-09,-7.056898e-08,-5.980526e-09,-6.532812e-08,-1.905857e-08,-2.170809e-08,-4.357428e-08,3.977849e-08,-5.393176e-09
3,3,-8.315076,7.467424,2.522501,-0.738182,-1.972197,-4.542781,-5.488337,-5.510764,-0.634931,...,-1.013841e-09,-2.702960e-08,-2.243326e-08,-7.728475e-08,1.846366e-08,-2.238339e-08,3.255535e-09,1.145492e-07,-4.428779e-08,3.383024e-08
4,4,0.698024,-0.125580,-0.109485,-1.899794,-3.550294,-1.752222,2.062500,0.609237,-1.256715,...,-7.700954e-08,7.565239e-10,3.397826e-08,8.835645e-08,-1.104942e-07,1.256305e-08,3.204138e-08,4.463313e-08,-5.343759e-08,-1.396384e-08


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,7.502268,-10.692819,-0.404433,-1.043061,-3.365657,7.976577,-0.905798,0.075209,2.911449,-0.346325,...,1.057144e-07,-1.960831e-08,3.438281e-08,-2.237564e-08,-2.577251e-08,3.101031e-08,4.925576e-08,1.060432e-08,-5.321290e-08,-2.239432e-08
1,2.932372,-1.768247,-1.257079,-2.001190,-2.707743,4.679531,-1.493178,-1.619118,2.283399,-2.251126,...,3.790522e-08,-2.500214e-08,-2.884480e-08,-1.528578e-08,-6.651166e-08,5.620558e-08,-1.265269e-08,-7.747284e-08,-7.012600e-08,3.852026e-09
2,-3.811744,2.214510,5.168598,-1.560522,-4.774682,0.298724,1.614868,-0.184465,-2.296425,-1.970529,...,3.322145e-08,-7.995277e-09,-7.056898e-08,-5.980526e-09,-6.532812e-08,-1.905857e-08,-2.170809e-08,-4.357428e-08,3.977849e-08,-5.393176e-09
3,-8.315076,7.467424,2.522501,-0.738182,-1.972197,-4.542781,-5.488337,-5.510764,-0.634931,1.845368,...,-1.013841e-09,-2.702960e-08,-2.243326e-08,-7.728475e-08,1.846366e-08,-2.238339e-08,3.255535e-09,1.145492e-07,-4.428779e-08,3.383024e-08
4,0.698024,-0.125580,-0.109485,-1.899794,-3.550294,-1.752222,2.062500,0.609237,-1.256715,2.859166,...,-7.700954e-08,7.565239e-10,3.397826e-08,8.835645e-08,-1.104942e-07,1.256305e-08,3.204138e-08,4.463313e-08,-5.343759e-08,-1.396384e-08


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.36892870113012954


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5,class_weight = "balanced" )

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.458197,0.458197,0.542123,0.452751,0.282443,0.282443,0.338314,0.278564


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    1.0000    0.9986       361
           2     0.9982    0.9982    0.9982       553
           3     1.0000    0.9967    0.9984       306

    accuracy                         0.9984      1220
   macro avg     0.9985    0.9983    0.9984      1220
weighted avg     0.9984    0.9984    0.9984      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2353    0.0764    0.1154       157
           2     0.4525    0.8650    0.5942       237
           3     0.3000    0.0462    0.0800       130

    accuracy                         0.4256       524
   macro avg     0.3293    0.3292    0.2632       524
weighted avg     0.3496    0.4256    0.3232       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.458197,0.458197,0.542123,0.452751,0.282443,0.282443,0.338314,0.278564
1,RandomForest,0.998361,0.998361,0.998363,0.998361,0.425573,0.425573,0.349605,0.323171


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5333    0.3989    0.4564       361
           2     0.5397    0.7866    0.6402       553
           3     0.4722    0.2222    0.3022       306

    accuracy                         0.5303      1220
   macro avg     0.5151    0.4692    0.4663      1220
weighted avg     0.5209    0.5303    0.5010      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3077    0.2293    0.2628       157
           2     0.4386    0.6329    0.5181       237
           3     0.2462    0.1231    0.1641       130

    accuracy                         0.3855       524
   macro avg     0.3308    0.3284    0.3150       524
weighted avg     0.3516    0.3855    0.3538       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.568668,0.510934,0.416031,0.416031,0.399968,0.392289
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.427481,0.427481,0.355380,0.329345
2,Adaboost,0.530328,0.530328,0.520892,0.501037,0.385496,0.385496,0.351632,0.353791


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9500    0.7895    0.8623       361
           2     0.7793    0.9964    0.8746       553
           3     0.9812    0.6830    0.8054       306

    accuracy                         0.8566      1220
   macro avg     0.9035    0.8230    0.8474      1220
weighted avg     0.8805    0.8566    0.8536      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2500    0.1656    0.1992       157
           2     0.4302    0.6498    0.5176       237
           3     0.2097    0.1000    0.1354       130

    accuracy                         0.3683       524
   macro avg     0.2966    0.3051    0.2841       524
weighted avg     0.3215    0.3683    0.3274       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.568668,0.510934,0.416031,0.416031,0.399968,0.392289
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.427481,0.427481,0.355380,0.329345
2,Adaboost,0.530328,0.530328,0.520892,0.501037,0.385496,0.385496,0.351632,0.353791
3,GBM,0.856557,0.856557,0.880478,0.853613,0.368321,0.368321,0.321484,0.327416


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4289    0.4681    0.4477       361
           2     0.5670    0.3671    0.4457       553
           3     0.3205    0.4902    0.3876       306

    accuracy                         0.4279      1220
   macro avg     0.4388    0.4418    0.4270      1220
weighted avg     0.4643    0.4279    0.4317      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2431    0.2803    0.2604       157
           2     0.4236    0.2574    0.3202       237
           3     0.2161    0.3308    0.2614       130

    accuracy                         0.2824       524
   macro avg     0.2943    0.2895    0.2807       524
weighted avg     0.3180    0.2824    0.2877       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.458197,0.458197,0.542123,0.452751,0.282443,0.282443,0.338314,0.278564
1,RandomForest,0.998361,0.998361,0.998363,0.998361,0.425573,0.425573,0.349605,0.323171
2,SVM,0.427869,0.427869,0.464340,0.431697,0.282443,0.282443,0.318038,0.287686


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5028    0.4986    0.5007       361
           2     0.5777    0.7125    0.6381       553
           3     0.4889    0.2876    0.3621       306

    accuracy                         0.5426      1220
   macro avg     0.5231    0.4996    0.5003      1220
weighted avg     0.5333    0.5426    0.5282      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2774    0.2739    0.2756       157
           2     0.4539    0.5401    0.4933       237
           3     0.2874    0.1923    0.2304       130

    accuracy                         0.3740       524
   macro avg     0.3396    0.3354    0.3331       524
weighted avg     0.3597    0.3740    0.3628       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.568668,0.510934,0.416031,0.416031,0.399968,0.392289
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.427481,0.427481,0.355380,0.329345
2,Adaboost,0.530328,0.530328,0.520892,0.501037,0.385496,0.385496,0.351632,0.353791
3,GBM,0.856557,0.856557,0.880478,0.853613,0.368321,0.368321,0.321484,0.327416
4,SVM,0.457377,0.457377,0.583255,0.303216,0.448473,0.448473,0.263846,0.283688
5,KNN,0.542623,0.542623,0.533265,0.528206,0.374046,0.374046,0.359705,0.362846


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4309    0.2161    0.2878       361
           2     0.4814    0.8174    0.6059       553
           3     0.3100    0.1013    0.1527       306

    accuracy                         0.4598      1220
   macro avg     0.4074    0.3782    0.3488      1220
weighted avg     0.4235    0.4598    0.3981      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2133    0.1019    0.1379       157
           2     0.4474    0.7722    0.5666       237
           3     0.3500    0.1077    0.1647       130

    accuracy                         0.4065       524
   macro avg     0.3369    0.3273    0.2897       524
weighted avg     0.3531    0.4065    0.3384       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.538525,0.538525,0.568668,0.510934,0.416031,0.416031,0.399968,0.392289
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.427481,0.427481,0.355380,0.329345
2,Adaboost,0.530328,0.530328,0.520892,0.501037,0.385496,0.385496,0.351632,0.353791
3,GBM,0.856557,0.856557,0.880478,0.853613,0.368321,0.368321,0.321484,0.327416
4,SVM,0.457377,0.457377,0.583255,0.303216,0.448473,0.448473,0.263846,0.283688
5,KNN,0.542623,0.542623,0.533265,0.528206,0.374046,0.374046,0.359705,0.362846
6,GNB,0.459836,0.459836,0.423461,0.398110,0.406489,0.406489,0.353120,0.338440
